# Fix Boundary Condition Bug

In [8]:
import h5py
import pandas as pd
from pathlib import Path

In [16]:
hdf_path = Path(r'\\limno.com\files\AAO\AAOWorking\2ERDC12 - Clearwater\ClearwaterHECRAS_testCases\sumwereCreek_TSM_testing_timestep')
hdf_file_path = hdf_path / 'clearWaterTestCases.p49.hdf'
print(hdf_file_path)

\\limno.com\files\AAO\AAOWorking\2ERDC12 - Clearwater\ClearwaterHECRAS_testCases\sumwereCreek_TSM_testing_timestep\clearWaterTestCases.p49.hdf


## Open File

In [17]:
infile = h5py.File(hdf_file_path, 'r')

In [18]:
infile

<HDF5 file "clearWaterTestCases.p49.hdf" (mode r)>

## Find Boundaries

### Existing Code

In [19]:
hdf_paths = {
    'boundary_condition_external_faces': 'Geometry/Boundary Condition Lines/External Faces',
    'boundary_condition_attributes': 'Geometry/Boundary Condition Lines/Attributes/',
    'boundary_condition_fixes': 'Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Boundary Conditions'
}

In [24]:
external_faces = pd.DataFrame(infile[hdf_paths['boundary_condition_external_faces']][()])
attributes = pd.DataFrame(infile[hdf_paths['boundary_condition_attributes']][()])

In [21]:
print(external_faces)

     BC Line ID  Face Index  FP Start Index  FP End Index  Station Start  \
0             0         954            6792          4461      -7.063520   
1             0       11811            4461          4462      24.761791   
2             0        9564            4462          5493      44.808739   
3             0       13046            5493          1080      64.855682   
4             0         109            1080          1076      83.864265   
..          ...         ...             ...           ...            ...   
265           3        3876            1172          1171       0.000000   
266           3        3881            1171          1199       0.000000   
267           3        3901            1199          1220       0.000000   
268           3        4093            1220          1267       0.000000   
269           3        3802            1267          1323      17.004597   

     Station End  
0      24.761791  
1      44.808739  
2      64.855682  
3      83.8

In [22]:
print(attributes)

                         Name           SA-2D         Type      Length
0                  b'US_Flow'  b'Perimeter 1'  b'External'  364.667389
1                 b'DS_Stage'  b'Perimeter 1'  b'External'  590.738464
2             b'PwrPlnt_Flow'  b'Perimeter 1'  b'External'   83.459290
3  b'CoriolisColdSpringCreek'  b'Perimeter 1'  b'External'   28.750957


In [25]:
list_of_boundaries = infile[hdf_paths['boundary_condition_fixes'][()]]

TypeError: string indices must be integers, not 'tuple'

In [35]:
str_df = attributes.select_dtypes([object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    attributes[col] = str_df[col]
boundary_attributes = attributes

In [37]:
str_df

,Name,SA-2D,Type
0,US_Flow,Perimeter 1,External
1,DS_Stage,Perimeter 1,External
2,PwrPlnt_Flow,Perimeter 1,External
3,CoriolisColdSpringCreek,Perimeter 1,External


In [36]:
boundary_attributes

,Name,SA-2D,Type,Length
0,US_Flow,Perimeter 1,External,364.667389
1,DS_Stage,Perimeter 1,External,590.738464
2,PwrPlnt_Flow,Perimeter 1,External,83.459290
3,CoriolisColdSpringCreek,Perimeter 1,External,28.750957


In [83]:
boundary_attributes['BC Line ID'] = boundary_attributes.index
boundary_data = pd.merge(external_faces, boundary_attributes, on = 'BC Line ID', how = 'left')
boundary_data

,BC Line ID,Face Index,FP Start Index,FP End Index,Station Start,Station End,Name,SA-2D,Type,Length
0,0,954,6792,4461,-7.063520,24.761791,US_Flow,Perimeter 1,External,364.667389
1,0,11811,4461,4462,24.761791,44.808739,US_Flow,Perimeter 1,External,364.667389
2,0,9564,4462,5493,44.808739,64.855682,US_Flow,Perimeter 1,External,364.667389
3,0,13046,5493,1080,64.855682,83.864265,US_Flow,Perimeter 1,External,364.667389
4,0,109,1080,1076,83.864265,103.902901,US_Flow,Perimeter 1,External,364.667389
...,...,...,...,...,...,...,...,...,...,...
265,3,3876,1172,1171,0.000000,0.000000,CoriolisColdSpringCreek,Perimeter 1,External,28.750957
266,3,3881,1171,1199,0.000000,0.000000,CoriolisColdSpringCreek,Perimeter 1,External,28.750957
267,3,3901,1199,1220,0.000000,0.000000,CoriolisColdSpringCreek,Perimeter 1,External,28.750957
268,3,4093,1220,1267,0.000000,17.004597,CoriolisColdSpringCreek,Perimeter 1,External,28.750957


### Updated Code

In [41]:
list_of_boundaries = list(infile[hdf_paths['boundary_condition_fixes']].keys())
list_of_boundaries

['CoriolisColdSpringCreek',
 'CoriolisColdSpringCreek - Flow per Face',
 'CoriolisColdSpringCreek - Stage per Face',
 'DS_Stage',
 'DS_Stage - Flow per Face',
 'DS_Stage - Stage per Face',
 'PwrPlnt_Flow',
 'PwrPlnt_Flow - Flow per Face',
 'PwrPlnt_Flow - Stage per Face',
 'US_Flow',
 'US_Flow - Flow per Face',
 'US_Flow - Stage per Face']

In [45]:
list(boundary_attributes.Name)

['US_Flow', 'DS_Stage', 'PwrPlnt_Flow', 'CoriolisColdSpringCreek']

In [89]:
# boundary_data = boundary_data_copy

In [118]:
df_ls = []
from clearwater_riverine.io.hdf import _parse_attributes

for boundary in list(boundary_attributes.Name):
    print(f'starting {boundary}')
    fpath = f"{hdf_paths['boundary_condition_fixes']}/{boundary} - Flow per Face"
    attrs = _parse_attributes(infile[fpath])
    boundary_faces_fix = attrs['Faces']
    boundary_faces_orig = boundary_data[(boundary_data.Name == boundary)]
    if set(boundary_faces_fix) != set(boundary_faces_orig['Face Index']):
        print(f'Extra boundary faces identified in HDF output for {boundary}.')
        diff = set(boundary_faces_orig['Face Index']) - set(boundary_faces_fix)
        print(f'Removing {diff}')
    df = boundary_data[(boundary_data.Name == boundary) & (boundary_data['Face Index'].isin(boundary_faces_fix))]
    df_ls.append(df)

starting US_Flow
starting DS_Stage
starting PwrPlnt_Flow
starting CoriolisColdSpringCreek
Extra boundary faces identified in HDF output for CoriolisColdSpringCreek.
Removing {7685, 3081, 2570, 7692, 1048, 540, 32, 11810, 11811, 39, 41, 48, 2097, 49, 567, 10814, 64, 10818, 69, 10823, 72, 10825, 591, 10832, 2131, 2132, 10837, 2136, 89, 11864, 10844, 10845, 3165, 608, 11879, 2666, 1130, 109, 621, 113, 635, 11900, 11902, 13956, 15493, 15494, 15495, 2184, 15497, 15496, 648, 142, 4249, 3225, 4253, 673, 677, 2215, 169, 1706, 3901, 2732, 2221, 2222, 15021, 180, 184, 198, 2766, 2767, 3279, 212, 1752, 1755, 220, 3806, 3295, 5855, 2789, 2790, 13046, 247, 250, 13051, 1276, 2301, 253, 1793, 1285, 1286, 264, 1804, 272, 2834, 277, 791, 280, 11033, 283, 9503, 9504, 11041, 288, 800, 3876, 2341, 11046, 11049, 809, 11051, 5931, 11053, 3374, 813, 303, 817, 3881, 819, 1844, 307, 3386, 315, 7484, 3388, 12606, 3390, 3392, 3393, 5951, 9539, 5953, 1854, 320, 319, 327, 12105, 3402, 9547, 3401, 3404, 9551, 336, 

In [106]:
df_full = pd.concat(df_ls)

In [107]:
df_full = df_full.drop(['Station Start', 'Station End'], axis=1)

In [108]:
df_full = df_full.drop_duplicates()

In [112]:
set([1, 2]) == set([2, 1])

True

In [119]:
infile.close()